1. Chuẩn bị Data

In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Reading ratings file
# Ignore the timestamp column
ratings = pd.read_csv('ratings.csv', encoding='latin-1', usecols=['userId', 'movieId', 'rating'])

# Reading users file
users = pd.read_csv('users.csv', sep='\t', encoding='latin-1', usecols=['user_id', 'gender', 'zipcode', 'age_desc', 'occ_desc'])

# Reading movies file
movies = pd.read_csv('movies.csv', encoding='latin-1', usecols=['movieId', 'title', 'genres'])

1.1 Ratings Dataset

In [28]:
# Check the top 5 rows
print(ratings.head())

   userId  movieId  rating
0       1        1     4.0
1       1        3     4.0
2       1        6     4.0
3       1       47     5.0
4       1       50     5.0


In [13]:
# Check the file info
print(ratings.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40022 entries, 0 to 40021
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   userId   40022 non-null  int64  
 1   movieId  40022 non-null  int64  
 2   rating   40022 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 938.1 KB
None


1.2. Users Dataset

In [29]:
# Check the top 5 rows
print(users.head())

   user_id gender zipcode  age_desc              occ_desc
0        1      F   48067  Under 18          K-12 student
1        2      M   70072       56+         self-employed
2        3      M   55117     25-34             scientist
3        4      M   02460     45-49  executive/managerial
4        5      M   55455     25-34                writer


In [30]:
# Check the file info
print(users.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6040 entries, 0 to 6039
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   6040 non-null   int64 
 1   gender    6040 non-null   object
 2   zipcode   6040 non-null   object
 3   age_desc  6040 non-null   object
 4   occ_desc  6040 non-null   object
dtypes: int64(1), object(4)
memory usage: 236.1+ KB
None


1.3. Movies Dataset

In [31]:
# Check the top 5 rows
print(movies.head())

   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  


In [32]:
# Check the file info
print(movies.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB
None


2. Thực hiện

Với tất cả những lý thuyết đó, tôi sẽ xây dựng một Công cụ đề xuất dựa trên nội dung để tính toán sự giống nhau giữa các bộ phim dựa trên thể loại phim. Nó sẽ gợi ý những bộ phim giống nhất với một bộ phim cụ thể dựa trên thể loại của nó. Để làm như vậy, tôi sẽ sử dụng tệp movies.csv.

In [44]:
# Break up the big genre string into a string array
movies['genres'] = movies['genres'].str.split('|')
# Convert genres to string value
movies['genres'] = movies['genres'].fillna("").astype('str')

Tôi không có số liệu định lượng để đánh giá hiệu suất của máy nên điều này sẽ phải được thực hiện một cách định tính. Để làm như vậy, tôi sẽ sử dụng hàm TfidfVectorizer từ scikit-learning, hàm này chuyển đổi văn bản thành các vectơ đặc trưng có thể được sử dụng làm đầu vào cho công cụ ước tính.


In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(movies['genres'])
tfidf_matrix.shape


(9742, 177)

Tôi sẽ sử dụng Độ tương tự Cosine để tính toán một đại lượng biểu thị độ giống nhau giữa hai bộ phim. Vì chúng tôi đã sử dụng TF-IDF Vectorizer, nên việc tính toán Sản phẩm chấm sẽ trực tiếp cung cấp cho chúng tôi Điểm tương đồng Cosine. Do đó, chúng tôi sẽ sử dụng linear_kernel của sklearn thay vì cosine_similarities vì nó nhanh hơn nhiều.

In [35]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim[:4, :4]

array([[1.        , 0.31379419, 0.0611029 , 0.05271111],
       [0.31379419, 1.        , 0.        , 0.        ],
       [0.0611029 , 0.        , 1.        , 0.35172407],
       [0.05271111, 0.        , 0.35172407, 1.        ]])

Bây giờ tôi có một ma trận tương tự cosine theo cặp cho tất cả các phim trong tập dữ liệu. Bước tiếp theo là viết một hàm trả về 20 phim giống nhau nhất dựa trên điểm tương tự cosine.

In [36]:
# Build a 1-dimensional array with movie titles
titles = movies['title']
indices = pd.Series(movies.index, index=movies['title'])

# Function that get movie recommendations based on the cosine similarity score of movie genres
def genre_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:21]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

Hãy thử và nhận các đề xuất hàng đầu cho một vài bộ phim và xem các đề xuất đó tốt như thế nào.

In [38]:
genre_recommendations('Good Will Hunting (1997)').head(20)

24                            Leaving Las Vegas (1995)
27                                   Persuasion (1995)
42                How to Make an American Quilt (1995)
45                        When Night Is Falling (1995)
66                                 Bed of Roses (1996)
75     Once Upon a Time... When We Were Colored (1995)
76                           Angels and Insects (1995)
93               Bridges of Madison County, The (1995)
115                       Up Close and Personal (1996)
151                                    Mad Love (1995)
162                         Scarlet Letter, The (1995)
170                               Total Eclipse (1995)
175                       Walk in the Clouds, A (1995)
183                              Before Sunrise (1995)
189                           Circle of Friends (1995)
213                            Immortal Beloved (1994)
232                                 Love Affair (1994)
261                  Picture Bride (Bijo photo) (1994)
262       

In [39]:
genre_recommendations('Toy Story (1995)').head(20)

1706                                          Antz (1998)
2355                                   Toy Story 2 (1999)
2809       Adventures of Rocky and Bullwinkle, The (2000)
3000                     Emperor's New Groove, The (2000)
3568                                Monsters, Inc. (2001)
6194                                     Wild, The (2006)
6486                               Shrek the Third (2007)
6948                       Tale of Despereaux, The (2008)
7760    Asterix and the Vikings (AstÃ©rix et les Vikin...
8219                                         Turbo (2013)
8927                             The Good Dinosaur (2015)
9430                                         Moana (2016)
5490    Twelve Tasks of Asterix, The (Les douze travau...
6448           TMNT (Teenage Mutant Ninja Turtles) (2007)
8357                                The Lego Movie (2014)
3194                                         Shrek (2001)
7530                               Gnomeo & Juliet (2011)
7805        Pu

In [43]:
genre_recommendations('Saving Private Ryan (1998)').head(20)

909                           Apocalypse Now (1979)
933                    Boot, Das (Boat, The) (1981)
1407          All Quiet on the Western Front (1930)
1503                     Saving Private Ryan (1998)
1826                      Thin Red Line, The (1998)
2216                        Dirty Dozen, The (1967)
2310                        Longest Day, The (1962)
2313                       Tora! Tora! Tora! (1970)
2573                                Red Dawn (1984)
2631                  Force 10 from Navarone (1978)
2716                   Fighting Seabees, The (1944)
2808                            Patriot, The (2000)
3610                      Behind Enemy Lines (2001)
3646                         Black Hawk Down (2001)
3724                        We Were Soldiers (2002)
3856                             Windtalkers (2002)
4114                                    Zulu (1964)
4121      Victory (a.k.a. Escape to Victory) (1981)
4243                       Gods and Generals (2003)
5009    Alex

Có thể thấy, tôi có một danh sách đề xuất kha khá cho Good Will Hunting (Phim chính kịch), Câu chuyện đồ chơi (Hoạt hình, Thiếu nhi, Hài) và Giải cứu binh nhì Ryan (Hành động, Kinh dị, Chiến tranh).

Nhìn chung, đây là những ưu điểm của việc sử dụng đề xuất dựa trên nội dung:

Không cần dữ liệu về người dùng khác, do đó không có vấn đề khởi động nguội hoặc thưa thớt.
Có thể giới thiệu cho người dùng với thị hiếu độc đáo.
Có thể giới thiệu các mặt hàng mới và không phổ biến.
Có thể cung cấp giải thích cho các mục được đề xuất bằng cách liệt kê các tính năng nội dung khiến một mục được đề xuất (trong trường hợp này là thể loại phim)
Tuy nhiên, có một số nhược điểm của việc sử dụng phương pháp này:

Tìm kiếm các tính năng thích hợp là khó khăn.
Không đề xuất các mục bên ngoài hồ sơ nội dung của người dùng.
Không thể khai thác đánh giá chất lượng của người dùng khác.